# 🎓 Pipeline de Treinamento do Modelo LSTM

Este notebook demonstra o **pipeline completo** de treinamento do modelo LSTM para previsão de ações.

## 📚 O que você vai aprender

1. Como carregar e preparar dados de ações
2. Como criar features para o modelo
3. Como dividir dados para treino/validação/teste
4. Como treinar o modelo LSTM
5. Como avaliar e salvar o modelo

---

## Passo 1: Configuração do Ambiente


In [2]:
import sys
from pathlib import Path

# Adicionar o diretório raiz ao path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Nossos módulos
from src.data.data_loader import StockDataLoader
from src.data.preprocessor import StockDataPreprocessor
from src.training.improved_trainer import ImprovedModelTrainer

print("✅ Ambiente configurado!")


✅ Ambiente configurado!


## Passo 2: Carregar Dados de Ações

Usamos a biblioteca `yfinance` para baixar dados históricos de ações.

**Por que dados pós-2021?**
- O crash de COVID em 2020 foi um evento atípico
- Treinar com dados pós-COVID resulta em modelos mais estáveis


In [3]:
# Configurações
SYMBOL = "AAPL"  # Ticker da Apple
START_DATE = "2021-01-01"  # Dados pós-COVID
END_DATE = datetime.now().strftime("%Y-%m-%d")

print(f"📊 Carregando dados de {SYMBOL}")
print(f"📅 Período: {START_DATE} até {END_DATE}")

# Carregar dados
loader = StockDataLoader()
df = loader.load_stock_data(SYMBOL, START_DATE, END_DATE)

print(f"✅ Carregados {len(df)} registros")
df.head()


2025-12-03 19:38:39.316 | INFO     | src.data.data_loader:load_stock_data:39 - Downloading data for AAPL from 2021-01-01 to 2025-12-03


📊 Carregando dados de AAPL
📅 Período: 2021-01-01 até 2025-12-03


2025-12-03 19:38:40.665 | INFO     | src.data.data_loader:load_stock_data:91 - Successfully loaded 1235 records for AAPL


✅ Carregados 1235 registros


Price,timestamp,close,high,low,open,volume,year,month,day,day_of_week,symbol
0,2021-01-04,125.974457,130.062953,123.394807,129.975346,143301900,2021,1,4,0,AAPL
1,2021-01-05,127.531982,128.242613,125.020474,125.468268,97664900,2021,1,5,1,AAPL
2,2021-01-06,123.239067,127.570935,123.024906,124.329336,155088000,2021,1,6,2,AAPL
3,2021-01-07,127.444374,128.135532,124.465612,124.952339,109578200,2021,1,7,3,AAPL
4,2021-01-08,128.544403,129.109007,126.772712,128.914305,105158200,2021,1,8,4,AAPL


## Passo 3: Feature Engineering

Criamos features adicionais que ajudam o modelo a aprender padrões:

| Feature | Descrição |
|---------|-----------|
| **ma_7, ma_30, ma_90** | Médias móveis de 7, 30 e 90 dias |
| **volatility_7, volatility_30** | Desvio padrão dos retornos |
| **momentum** | Tendência de preço |
| **roc_7, roc_30** | Rate of Change |
| **volume_ma_7** | Volume normalizado |


In [4]:
# Criar preprocessador com sequência de 60 dias
preprocessor = StockDataPreprocessor(sequence_length=60)

# Transformar dados em sequências
X, y, df_processed = preprocessor.fit_transform(df)

print(f"📊 Formato dos dados:")
print(f"   X (features): {X.shape}")
print(f"   y (targets): {y.shape}")
print(f"\n📝 Interpretação:")
print(f"   - {X.shape[0]} amostras")
print(f"   - {X.shape[1]} dias por sequência")
print(f"   - {X.shape[2]} features por dia")


2025-12-03 19:39:19.520 | INFO     | src.data.preprocessor:fit_transform:141 - Created 1175 sequences with shape (1175, 60, 16)


📊 Formato dos dados:
   X (features): (1175, 60, 16)
   y (targets): (1175,)

📝 Interpretação:
   - 1175 amostras
   - 60 dias por sequência
   - 16 features por dia


## Passo 4: Treinar o Modelo

O `ImprovedModelTrainer` inclui várias técnicas avançadas:

- **Early Stopping**: Para quando o modelo não melhora mais
- **Learning Rate Scheduler**: Reduz LR automaticamente
- **Gradient Clipping**: Evita explosão de gradientes
- **Attention Mechanism**: Foca nas partes importantes da sequência


In [5]:
# Criar trainer com configurações
trainer = ImprovedModelTrainer(
    symbol=SYMBOL,
    epochs=80,           # Máximo de épocas
    patience=10,         # Parar se não melhorar em 10 épocas
    batch_size=32,
    hidden_size=64,      # Tamanho da camada oculta
    num_layers=3,        # 3 camadas LSTM
    dropout=0.3          # 30% dropout
)

# Executar pipeline completo de treinamento
print("🚀 Iniciando treinamento...")
print("   Isso pode demorar alguns minutos.\n")

metrics = trainer.run_training_pipeline(
    start_date=START_DATE,
    end_date=END_DATE,
    train_split=0.7,
    val_split=0.15
)

print("\n✅ Treinamento concluído!")


2025-12-03 19:39:35.626 | INFO     | src.training.improved_trainer:run_training_pipeline:403 - ============================================================
2025-12-03 19:39:35.631 | INFO     | src.training.improved_trainer:run_training_pipeline:404 - 🎯 Treinando modelo para AAPL
2025-12-03 19:39:35.636 | INFO     | src.training.improved_trainer:run_training_pipeline:405 - 📅 Período: 2021-01-01 até 2025-12-03
2025-12-03 19:39:35.638 | INFO     | src.training.improved_trainer:run_training_pipeline:406 - ============================================================
2025-12-03 19:39:35.639 | INFO     | src.training.improved_trainer:run_training_pipeline:409 - 📥 Carregando dados...
2025-12-03 19:39:35.642 | INFO     | src.data.data_loader:load_stock_data:39 - Downloading data for AAPL from 2021-01-01 to 2025-12-03
2025-12-03 19:39:35.779 | INFO     | src.data.data_loader:load_stock_data:91 - Successfully loaded 1235 records for AAPL
2025-12-03 19:39:35.784 | INFO     | src.data.data_loader:v

🚀 Iniciando treinamento...
   Isso pode demorar alguns minutos.



2025-12-03 19:39:39.086 | INFO     | src.training.improved_trainer:fit:185 - 🚀 Iniciando treinamento: 80 épocas, batch=32
2025-12-03 19:39:39.087 | INFO     | src.training.improved_trainer:fit:186 - 📊 Dados: 822 treino, 176 validação
2025-12-03 19:39:40.440 | INFO     | src.training.improved_trainer:fit:212 - ✅ Epoch 1: Nova melhor val_loss=0.155466
2025-12-03 19:39:41.663 | INFO     | src.training.improved_trainer:fit:212 - ✅ Epoch 2: Nova melhor val_loss=0.131209
2025-12-03 19:39:42.830 | INFO     | src.training.improved_trainer:fit:212 - ✅ Epoch 3: Nova melhor val_loss=0.109055
2025-12-03 19:39:43.958 | INFO     | src.training.improved_trainer:fit:212 - ✅ Epoch 4: Nova melhor val_loss=0.016173
2025-12-03 19:39:45.086 | INFO     | src.training.improved_trainer:fit:212 - ✅ Epoch 5: Nova melhor val_loss=0.009406
2025-12-03 19:39:48.638 | INFO     | src.training.improved_trainer:fit:212 - ✅ Epoch 8: Nova melhor val_loss=0.002140
2025-12-03 19:39:50.856 | INFO     | src.training.improved


✅ Treinamento concluído!


## Passo 5: Analisar Resultados

Vamos interpretar as métricas do modelo treinado.


In [7]:
print("📊 MÉTRICAS DO MODELO")
print("=" * 40)
rmse = metrics.get('rmse', 0)
mae = metrics.get('mae', 0)
mape = metrics.get('mape', 0)
r2 = metrics.get('r2', 0)
directional_acc = metrics.get('directional_accuracy', 0)

print(f"   RMSE (Root Mean Square Error): ${rmse:.2f}")
print("     - Mede o erro médio quadrático entre as previsões e valores reais.")
print("     - Quanto menor, melhor. Muito usado para avaliar previsões de preços.")
print(f"   MAE (Mean Absolute Error): ${mae:.2f}")
print("     - Erro médio absoluto. Representa o erro médio real (em dólares) entre previsão e valor real.")
print("     - Quanto menor, melhor.")

print(f"   MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
print("     - Erro percentual médio absoluto. Indica o erro médio em relação ao valor real, em %.")
print("     - Quanto menor, melhor. <5% = excelente, <10% = bom, <20% = razoável.")

print(f"   R² (Coeficiente de Determinação): {r2:.4f}")
print("     - Mede o quanto as previsões explicam a variação dos dados reais. Varia de -∞ a 1.")
print("     - Quanto mais próximo de 1, melhor. Valores negativos indicam ajuste ruim.")

print(f"   Acurácia Direcional: {directional_acc:.2f}%")
print("     - Mede quantas vezes o modelo acertou a direção do movimento (subida/queda) em relação ao real.")
print("     - >50% geralmente é melhor que aleatório; quanto maior, melhor.")
print("=" * 40)

# Interpretação automática
if mape < 5:
    print("🏆 MAPE Excelente (<5%)!")
elif mape < 10:
    print("✅ MAPE Bom (5-10%)")
elif mape < 20:
    print("📊 MAPE Aceitável (10-20%)")
else:
    print("⚠️ MAPE precisa melhoria (>20%)")


📊 MÉTRICAS DO MODELO
   RMSE (Root Mean Square Error): $24.73
     - Mede o erro médio quadrático entre as previsões e valores reais.
     - Quanto menor, melhor. Muito usado para avaliar previsões de preços.
   MAE (Mean Absolute Error): $22.71
     - Erro médio absoluto. Representa o erro médio real (em dólares) entre previsão e valor real.
     - Quanto menor, melhor.
   MAPE (Mean Absolute Percentage Error): 9.75%
     - Erro percentual médio absoluto. Indica o erro médio em relação ao valor real, em %.
     - Quanto menor, melhor. <5% = excelente, <10% = bom, <20% = razoável.
   R² (Coeficiente de Determinação): 0.1555
     - Mede o quanto as previsões explicam a variação dos dados reais. Varia de -∞ a 1.
     - Quanto mais próximo de 1, melhor. Valores negativos indicam ajuste ruim.
   Acurácia Direcional: 49.43%
     - Mede quantas vezes o modelo acertou a direção do movimento (subida/queda) em relação ao real.
     - >50% geralmente é melhor que aleatório; quanto maior, melhor.